# 主要概念
Chatbot 是一個透過程式自動回覆的機制。

1. 先申請 Line@官方帳號
2. 申請 ngrok 的反向代理機制，取得一組 xxxxx.ngrok.io 的網址，以便作為 webhook 使用
3. 測試期間: 本機使用 flask 來建立簡易 web server  
    參考文件: https://xiaosean.github.io/server/2018-04-18-Flask_Ngrok/

4. 若要正式上線: 則應使用 heroku 之類的代理伺服器，或者 google app engine 看似也可以。
5. 測試完成之後再部屬到 heroku 或 GAE


## LINE 相關處理

參考網址: https://xiaosean.github.io/chatbot/2018-04-10-LineChatbot/

1.至line申請develop帳號  我們要進入Line Developer https://developers.line.biz/en/  

create new provider  

create new channel 選擇 messaging API

new channel 要用 chrome 建立。appname 不能有底線

建立完畢到 channel 內找到 QR code 加入好友，才能開始測試。

Channel ID 
1618627869

Channel secret 
12cd193da30c66d6debbff7ed6fd0e0c

Basic ID
@043yqfiq

Other
Your user ID 
U59c72931d3e724aa6bc757c66b83afb3

Channel access token (long-lived) 

OeUcBaukfBDjXEukC3bvqOzvA/2NRBPNgxv1RGuVCgn93HE7ih0RZlwGVfav+hge/X2otNc2NGHA40blHg6bctERa8e6Z2KMAvfKzGKkH/BXa17/vnZya3KzpNJCj8fTet3r2WmLW7aiz949bSL8ugdB04t89/1O/w1cDnyilFU=

開啟 webhook
https://cc57e528.ngrok.io/callback


安裝
pip install line-bot-sdk


最後要 登入確定 webhook 設定有打開。

![xxxx](images/webhook_enable.png)



2.結合Line Bot SDK Github  
3.可用Line Chatbot複誦我傳過去的訊息  


## 底下是一個 flask 範例程式，直接回覆收到的訊息

存檔成 flask_linebot.py 執行

    python flask_linebot.py 

即可啟動， flask 預設在 :5000 的位置。


In [ ]:
from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)

app = Flask(__name__)

YOUR_CHANNEL_ACCESS_TOKEN = 'OeUcBaukfBDjXEukC3bvqOzvA/2NRBPNgxv1RGuVCgn93HE7ih0RZlwGVfav+hge/X2otNc2NGHA40blHg6bctERa8e6Z2KMAvfKzGKkH/BXa17/vnZya3KzpNJCj8fTet3r2WmLW7aiz949bSL8ugdB04t89/1O/w1cDnyilFU='
YOUR_CHANNEL_SECRET = '12cd193da30c66d6debbff7ed6fd0e0c'


line_bot_api = LineBotApi(YOUR_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(YOUR_CHANNEL_SECRET)


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    textin = event.message.text

    if '好嗎' in textin:
        textout = '很好'
    elif '天氣' in textin:
        textout = '今天天氣很好'
    else:
        textout = '我不懂你說什麼，請再說一次'
    
    #textout = textin


    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=textout))


if __name__ == "__main__":
    app.run()


## ngrok 反向代理

到官網下載 ngrok 解壓後的 .exe 即可使用
登入帳號，取得 auth token 

    > ngrok authtoken 1QUysRUo97w5mdB6sCZvTTMM0aK_3unoMs6nYd7grgCkuhbj3
    > ngrok.exe http 5000

轉接到本機的 5000 port 與 flask 配合



![](images/ngrok.png)

# 快速安裝

    pip install flask
    pip install line-bot-sdk
    pip install BeautifulSoup4
    pip install lxml
    git clone https://github.com/jiangsir/108-PythonDataScience
    
    啟動 flask
    python flask_linebot.py 
    
    啟動反向代理
    ngrok authtoken 1QUysRUo97w5mdB6sCZvTTMM0aK_3unoMs6nYd7grgCkuhbj3
    ngrok.exe http 5000
    
    獲得反向代理網址
    進入 Line Developer 
    Line Developer https://developers.line.biz/en/
    設定 web hook https://cc57e528.ngrok.io/callback
    
    
    